In [1]:
# Import all required libraries
import pandas
from pandas import *
import numpy
from datetime import datetime

# Initialize values

sample_size_train = 0
enable_scaler=True
nb_models = 10

# Number of value on which to train, if null, train on all value

# -1 = 50% up, 50% down
sample_size_train = 200000
sample_internal_test = 10000

In [2]:
# Read training data + test data
df_data = pandas.read_csv("../input/application_train.csv")
df_test = pandas.read_csv("../input/application_test.csv")

df_data['TARGET'].mean()

0.080728819456864956

In [3]:
# Only select a small sample, faster local testing

# Choose randomly 1k value out of those
df_data_1 = df_data.loc[df_data['TARGET'] == 1]
df_data_0 = df_data.loc[df_data['TARGET'] == 0].sample(24825)

df_internal_test = pandas.concat([df_data_0, df_data_1])
df_internal_test = df_internal_test.sample(sample_internal_test)

print("Sample internal test = {0}".format(df_internal_test['TARGET'].count()))
# We should remove them from df_data now

df_train = df_data.sample(sample_size_train)
display(df_internal_test.sample(3))
display(df_train.sample(10))

Sample internal test = 10000


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
286267,431547,0,Cash loans,M,Y,N,0,225000.0,590337.0,28530.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
181104,309883,1,Cash loans,M,N,Y,1,135000.0,314055.0,21375.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
33076,138332,1,Cash loans,F,N,Y,0,121500.0,590337.0,16695.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
228493,364655,0,Cash loans,M,N,Y,0,346500.0,194076.0,17928.0,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,4.0
220650,355615,0,Cash loans,F,Y,Y,0,202500.0,521280.0,26743.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
90000,204497,0,Cash loans,M,Y,Y,0,76500.0,900000.0,26316.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
266160,408320,0,Cash loans,F,N,Y,0,135000.0,592560.0,31153.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
120804,240060,0,Cash loans,M,N,Y,0,99900.0,254700.0,24192.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
39746,146029,0,Cash loans,F,N,Y,0,135000.0,206280.0,10161.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0
107226,224406,0,Cash loans,M,N,Y,0,180000.0,355536.0,19417.5,...,0,0,0,0,0.0,0.0,1.0,0.0,0.0,4.0
139586,261847,0,Cash loans,M,Y,Y,0,144000.0,360000.0,18508.5,...,0,0,0,0,0.0,0.0,0.0,0.0,2.0,1.0
95340,210690,1,Cash loans,F,N,N,0,202500.0,1260000.0,40644.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
17259,120129,0,Cash loans,F,N,N,0,53613.0,183384.0,10656.0,...,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
# Preparing all data
def prepare_data(df, dummies = []):
    df = pandas.get_dummies(df, prefix=dummies, 
                            columns=dummies)
    
    # Should we give more information? Like nb sales previous X weeks?
    # Previous sales of last 2/3 year at the same date (bank holiday, black friday, etc.)
    # Probably yes as this information is available, would be a good try
    return df

dummies = ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',
                                'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE',
                               'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE']
print(dummies)

df_train = prepare_data(df_train, dummies)
df_test = prepare_data(df_test, dummies)
df_internal_test = prepare_data(df_internal_test, dummies)

for column in df_train.columns:
    if column not in df_test.columns:
        df_test[column] = 0
    if column not in df_internal_test.columns:
        df_internal_test[column] = 0

for column in df_test.columns:
    if column not in df_train.columns:
        df_train[column] = 0
    if column not in df_internal_test.columns:
        df_internal_test[column] = 0

for column in df_internal_test.columns:
    if column not in df_train.columns:
        df_train[column] = 0
    if column not in df_test.columns:
        df_test[column] = 0
        

# Order the columns so that they all have the same
df_test = df_test[df_train.columns]
df_internal_test = df_internal_test[df_train.columns]

['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE']


In [5]:
# Generate our training/validation datasets
from sklearn import model_selection

# Name of the result column
result_cols = ['TARGET']
result_excl_cols = 'TARGET_'

input_cols = []
for dummy in dummies:
    input_cols.append("{0}_".format(dummy))

# Then, we add some columns based on the feeling
final_cols = ["DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH",
             #"AMT_GOODS_PRICE", "AMT_ANNUITY", 
              "AMT_CREDIT", "AMT_INCOME_TOTAL", "CNT_CHILDREN"]

#final_cols = ["DAYS_BIRTH"]
input_cols = input_cols + final_cols


# Get the final values
def get_values(df, cols=[], excl_cols = "doqwidjoqwidjqwoidjqwoidjqwodijqw"):
    columns = df.columns.values
    # Remove all columns that are not inside the list
    for column in columns:
        find = False
        if column.startswith(excl_cols):
            print("Ignoring {0}".format(column))
        else:
            for col in cols:
                if column.startswith(col):
                    find = True
        if not find:
            df = df.drop(columns=[column])
        else:
            df[column] = df[column].fillna(value=0)
    new_order = sorted(df.columns.values)
    print(new_order)
    # Same order for both training and testing set
    df = df[new_order]
    return df.values

print(input_cols)
X_train = get_values(df_train, input_cols)
y_train = get_values(df_train, result_cols, result_excl_cols).ravel()

X_test = get_values(df_internal_test, input_cols)
y_test = get_values(df_internal_test, result_cols, result_excl_cols).ravel()

X_final_test = get_values(df_test, input_cols)




['NAME_CONTRACT_TYPE_', 'CODE_GENDER_', 'FLAG_OWN_CAR_', 'FLAG_OWN_REALTY_', 'NAME_TYPE_SUITE_', 'NAME_INCOME_TYPE_', 'NAME_EDUCATION_TYPE_', 'NAME_FAMILY_STATUS_', 'NAME_HOUSING_TYPE_', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'CNT_CHILDREN']
['AMT_CREDIT', 'AMT_INCOME_TOTAL', 'CNT_CHILDREN', 'CODE_GENDER_F', 'CODE_GENDER_M', 'CODE_GENDER_XNA', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'DAYS_REGISTRATION', 'FLAG_OWN_CAR_N', 'FLAG_OWN_CAR_Y', 'FLAG_OWN_REALTY_N', 'FLAG_OWN_REALTY_Y', 'NAME_CONTRACT_TYPE_Cash loans', 'NAME_CONTRACT_TYPE_Revolving loans', 'NAME_EDUCATION_TYPE_Academic degree', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_EDUCATION_TYPE_Incomplete higher', 'NAME_EDUCATION_TYPE_Lower secondary', 'NAME_EDUCATION_TYPE_Secondary / secondary special', 'NAME_FAMILY_STATUS_Civil marriage', 'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_S

In [6]:
final_cols = ["DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH",
             #"AMT_GOODS_PRICE", "AMT_ANNUITY", 
              "AMT_CREDIT", "AMT_INCOME_TOTAL", "CNT_CHILDREN"]
for col in final_cols:
    print(col)
    print(df_train[[col]].isnull().values.any())


DAYS_BIRTH
False
DAYS_EMPLOYED
False
DAYS_REGISTRATION
False
DAYS_ID_PUBLISH
False
AMT_CREDIT
False
AMT_INCOME_TOTAL
False
CNT_CHILDREN
False


In [7]:
# Normalize the data
X_all = [x + y + z for x, y, z in zip(X_train, X_final_test, X_test)]
#print(len(X_all))
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler() 

# Don't cheat - fit only on training data
# Def adding x_train + X_test + X_validation to fit all of them

if enable_scaler:
    scaler.fit(X_all)  

    X_train = scaler.transform(X_train) 
    X_test = scaler.transform(X_test)
    X_final_test = scaler.transform(X_final_test) 

In [8]:
from sklearn.cross_validation import train_test_split

# Defining all those nice sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:
def get_comparator3000(Y_validation, Y_validation_predict):
    best_result = 0
    for r in range(0, 100):
        boundary = r/100.0
        result = 0
        for i in range(0, len(Y_validation)):
            if (Y_validation[i] == 1 and Y_validation_predict[i] > boundary) or (Y_validation[i] == 0 and Y_validation_predict[i] <= boundary):
                result += 1
        if result > best_result:
            best_result = result
            print("current best boundary = {0} with score {1}".format(boundary, best_result/ len(Y_validation)))
    return best_result / len(Y_validation)

In [10]:
# Import algorithm
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor

models = []

for i in range(5, 5 + nb_models):
    models.append(('MLPRegressor_adam_{0}'.format(i), MLPRegressor(hidden_layer_sizes=(8,),  activation='relu', solver='adam', alpha=0.001, batch_size='auto',
    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
    random_state=i, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)))
    
#    models.append(('MLPClassifier_adam_{0}'.format(i), MLPClassifier(hidden_layer_sizes=(8,),  activation='relu', solver='adam', alpha=0.001, batch_size='auto',
#    learning_rate='constant', learning_rate_init=0.01, power_t=0.5, max_iter=1000, shuffle=True,
#    random_state=i, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True,
#    early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)))

#models.append(('LR', LogisticRegression()))
#models.append(('LDA', LinearDiscriminantAnalysis()))
#models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
#models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC()))
    
# High value until first model get solved
max_score = 10000
best_model = "UNKNOWN"

res = []
submission = []
# Testing all models, one by one
for name, model in models:
    print("Executing for model {0}".format(name))
    time_start = datetime.now()

    # Training the model
    model.fit(X_train, y_train)
    
    print("Finish fit for {0}".format(name))

    y_test_result = model.predict(X_test)
    res.append(y_test_result)
    # We can calculate the avg error
    score = get_comparator3000(y_test, y_test_result)
    print("Model {0} got score of {1}, time: {2}".format(name, score, datetime.now() - time_start))
    
    
    # Doing as well on final one
    submission.append(model.predict(X_final_test))

Executing for model MLPRegressor_adam_5
Finish fit for MLPRegressor_adam_5
current best boundary = 0.0 with score 0.4954
current best boundary = 0.1 with score 0.5049
Model MLPRegressor_adam_5 got score of 0.5049, time: 0:00:02.494768
Executing for model MLPRegressor_adam_6
Finish fit for MLPRegressor_adam_6
current best boundary = 0.0 with score 0.4954
current best boundary = 0.09 with score 0.5047
Model MLPRegressor_adam_6 got score of 0.5047, time: 0:00:03.081388
Executing for model MLPRegressor_adam_7
Finish fit for MLPRegressor_adam_7
current best boundary = 0.0 with score 0.4954
current best boundary = 0.06 with score 0.6021
Model MLPRegressor_adam_7 got score of 0.6021, time: 0:00:03.274354
Executing for model MLPRegressor_adam_8
Finish fit for MLPRegressor_adam_8
current best boundary = 0.0 with score 0.4954
current best boundary = 0.1 with score 0.5046
Model MLPRegressor_adam_8 got score of 0.5046, time: 0:00:02.037290
Executing for model MLPRegressor_adam_9
Finish fit for MLP

In [11]:
# For all result in res, if test, display the result, if not, write it to a file
final_res = []
nb_variable = len(res[0])
for variable in range(0, nb_variable):
    final_res.append(0.0)
    for i in range(0, len(res)):
        final_res[variable] += res[i][variable]
    final_res[variable] = final_res[variable] / len(res)


# We can calculate the avg error
score = get_comparator3000(y_test, final_res)
print("avg model got score of {0}".format(score))


# Same for the submission
final_submission = []
nb_variable = len(submission[0])
for variable in range(0, nb_variable):
    final_submission.append(0.0)
    for i in range(0, len(submission)):
        final_submission[variable] += submission[i][variable]
    final_submission[variable] = final_submission[variable] / len(submission)
    if final_submission[variable] < 0.08:
        final_submission[variable] = 0.0
    else:
        final_submission[variable] = 1.0

print("Writing output file merged.csv".format(name))

df_test['TARGET'] = final_submission
result_df = df_test[['SK_ID_CURR', 'TARGET']]
result_df['TARGET'] = final_submission

result_df.to_csv("merged.csv".format(name), index=False)

current best boundary = 0.0 with score 0.4954
current best boundary = 0.06 with score 0.5032
current best boundary = 0.07 with score 0.5791
current best boundary = 0.08 with score 0.6062
avg model got score of 0.6062
Writing output file merged.csv


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
